[View in Colaboratory](https://colab.research.google.com/github/krithish1999/UD/blob/master/TensorFlowPredictingIris.ipynb)

In [1]:
from __future__ import absolute_import, division, print_function

import os
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.contrib.eager as tfe

tf.enable_eager_execution()

print("TensorFlow version: {}".format(tf.VERSION))
print("Eager execution: {}".format(tf.executing_eagerly()))


TensorFlow version: 1.8.0
Eager execution: True


In [2]:
train_dataset_url = "http://download.tensorflow.org/data/iris_training.csv"
train_dataset_fp = tf.keras.utils.get_file(fname=os.path.basename(train_dataset_url),origin=train_dataset_url)
print("Local copy of the dataset file: {}".format(train_dataset_fp))


8192/2194 [================================================================================================================] - 0s 0us/step
Local copy of the dataset file: /content/.keras/datasets/iris_training.csv


In [5]:
!head -n5 {train_dataset_fp}

120,4,setosa,versicolor,virginica
6.4,2.8,5.6,2.2,2
5.0,2.3,3.3,1.0,1
4.9,2.5,4.5,1.7,2
4.9,3.1,1.5,0.1,0


In [0]:
def parse_csv(line):
  example_defaults = [[0.],[0.],[0.],[0.],[0]]
  parsed_line = tf.decode_csv(line,example_defaults)
  features = tf.reshape(parsed_line[:-1],shape=(4,))
  label = tf.reshape(parsed_line[-1],shape=())
  return features,label

In [19]:
train_dataset = tf.data.TextLineDataset(train_dataset_fp)
train_dataset = train_dataset.skip(1)             
train_dataset = train_dataset.map(parse_csv)     
train_dataset = train_dataset.shuffle(buffer_size=1000)  
train_dataset = train_dataset.batch(32)
features , label = iter(train_dataset).next()
print("example features:", features[0])
print("example label:", label[0])





example features: tf.Tensor([6.3 2.5 5.  1.9], shape=(4,), dtype=float32)
example label: tf.Tensor(2, shape=(), dtype=int32)


In [0]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(10, activation="relu", input_shape=(4,)),  # input shape required
  tf.keras.layers.Dense(10, activation="relu"),
  tf.keras.layers.Dense(3)
])


In [0]:
def loss(model, x, y):
  y_ = model(x)
  return tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)


def grad(model, inputs, targets):
  with tf.GradientTape() as tape:
    loss_value = loss(model, inputs, targets)
  return tape.gradient(loss_value, model.variables)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)



In [22]:
## Note: Rerunning this cell uses the same model variables

# keep results for plotting
train_loss_results = []
train_accuracy_results = []

num_epochs = 201

for epoch in range(num_epochs):
  epoch_loss_avg = tfe.metrics.Mean()
  epoch_accuracy = tfe.metrics.Accuracy()

  # Training loop - using batches of 32
  for x, y in train_dataset:
    # Optimize the model
    grads = grad(model, x, y)
    optimizer.apply_gradients(zip(grads, model.variables),
                              global_step=tf.train.get_or_create_global_step())

    # Track progress
    epoch_loss_avg(loss(model, x, y))  # add current batch loss
    # compare predicted label to actual label
    epoch_accuracy(tf.argmax(model(x), axis=1, output_type=tf.int32), y)

  # end epoch
  train_loss_results.append(epoch_loss_avg.result())
  train_accuracy_results.append(epoch_accuracy.result())
  
  if epoch % 50 == 0:
    print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch,
                                                                epoch_loss_avg.result(),
                                                                epoch_accuracy.result()))


Epoch 000: Loss: 2.102, Accuracy: 30.000%
Epoch 050: Loss: 0.668, Accuracy: 64.167%
Epoch 100: Loss: 0.509, Accuracy: 69.167%
Epoch 150: Loss: 0.357, Accuracy: 96.667%
Epoch 200: Loss: 0.229, Accuracy: 97.500%
